In [47]:
import os 
import numpy as np

import sys
print(sys.path)

from deephive.environment.deephive_utils import *
from deephive.environment.utils import *

['/Users/elotech/Desktop/DeepHiveV2/notebooks', '/opt/homebrew/Cellar/python@3.11/3.11.7/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/opt/homebrew/Cellar/python@3.11/3.11.7/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/opt/homebrew/Cellar/python@3.11/3.11.7/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Users/elotech/.virtualenvs/mlEnv/lib/python3.11/site-packages', '/Users/elotech/Documents/CODES/array-scaler', '/Users/elotech/Desktop/DeepHiveV2/src', '/opt/homebrew/opt/python-tk@3.11/libexec']
Using device: cpu


In [48]:
# config_path = "config/config.json"
# # load config
# config = parse_config(config_path)

compare_num = 1
log_scale = True
all_symbols = ["-", "--", "-", "--", "-", "--"]
all_colors = ["r", "g", "b", "k", "m", "y"]



# load the .npy file in each experiment folder in the result_path

def compare_experiments(exp_list, result_path, titles, log_scale=False, compare_num=1):
    save_path = "/Users/elotech/Desktop/DeepHiveV2/experiments/results/comparisons/comparison_" + str(compare_num) +".png"

    symbol_list = []
    color_list = []
    label_list = []
    gbest_values = []
    for i, exp_num in enumerate(exp_list):
        exp_path = result_path + "experiment_" + str(exp_num) + "/"
        # load the .npy file
        gbest_values.append(np.load(exp_path + f"experiment_{exp_num}.npy") * -1)
        # get the label
        label_list.append(titles[i])
        # get the symbol
        symbol_list.append(all_symbols[i])
        # get the color
        color_list.append(all_colors[i])

    # plot the gbest_values
    plot_num_function_evaluation(fopt=np.array(gbest_values), label_list=label_list, symbol_list=symbol_list,
                                 color_list=color_list, save_dir=save_path,
                                 n_agents=10, opt_value=None, log_scale=log_scale)
    
    
# compare_experiments(exp_list, result_path, compare_num, titles, log_scale=log_scale)

In [55]:
result_path = "/Users/elotech/Desktop/DeepHiveV2/experiments/results_2/"
exp_list = [11, 12, 13, 14, 15, 16,17]
compare_num = 5
log_scale = False
#compare_experiments(exp_list=exp_list, result_path=result_path, titles=titles, log_scale=log_scale, compare_num=compare_num)

In [56]:
def load_json(config_path):
    with open(config_path, 'r') as f:
        config = json.load(f)
    return config

def mean_confidence_interval(data, confidence=0.95):
    # check if data is just a single array and reshape it to a 2D array
    if len(data.shape) == 1:
        data = data.reshape(1, -1)
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a, axis = 0), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [57]:
label_list = []
gbest_values = []
for i, exp_num in enumerate(exp_list):
    exp_path = result_path + "exp_" + str(exp_num) + "/"
    # load the .npy file
    gbest_values.append(np.load(exp_path + f"gbestVals.npy") * -1)
    # get the label
    summary_json = load_json(exp_path + "run_summary.json")
    title = summary_json["title"]
    label_list.append(title)

In [68]:
gbest_values[0][:, len(gbest_values[0][0])//2:]

array([[2957.98663343, 2957.98663343, 2957.98663343, ..., 2957.98663343,
        2957.98663343, 2938.29746153],
       [3020.72633759, 3020.72633759, 3020.72633759, ..., 3020.72633759,
        3020.72633759, 3020.72633759],
       [3011.28131056, 3011.28131056, 3011.28131056, ..., 3011.28131056,
        3011.28131056, 3011.28131056],
       ...,
       [2979.24125436, 2979.24125436, 2979.24125436, ..., 2979.24125436,
        2973.57949785, 2973.57949785],
       [3060.34638492, 3060.34638492, 3060.34638492, ..., 3060.34638492,
        3060.34638492, 3060.34638492],
       [3010.32639354, 3010.32639354, 3010.32639354, ..., 3004.83544707,
        3004.83544707, 3004.83544707]])

In [87]:
result_comparision = []
for i, gbest in enumerate(gbest_values):
    full_mean, full_lower, full_upper = mean_confidence_interval(gbest)
    mid_mean, mid_lower, mid_upper = mean_confidence_interval(gbest[:, :len(gbest[0])//2])
    result_comparision.append({
        "title": label_list[i],
        "mean": full_mean[-1],
        "lower": full_lower[-1],
        "upper": full_upper[-1],
        "mid_mean": mid_mean[-1],
        "mid_lower": mid_lower[-1],
        "mid_upper": mid_upper[-1]
    })
    #result_comparision[label_list[i]] = {"mean": mean[-1], "lower": lower[-1], "upper": upper[-1]}
    
    

In [88]:
result_comparision

[{'title': 'CEC17 Function 27 - NO SPLITTING-UNFREEZE',
  'mean': 2987.0136822308873,
  'lower': 2974.5782222357448,
  'upper': 2999.44914222603,
  'mid_mean': 3009.054367228565,
  'mid_lower': 2996.745468720921,
  'mid_upper': 3021.3632657362086},
 {'title': 'CEC17 Function 27 - NO SPLITTING-DELAYED DECAY-FREEZE',
  'mean': 2973.589800979984,
  'lower': 2965.058110316528,
  'upper': 2982.1214916434396,
  'mid_mean': 3003.0397821212105,
  'mid_lower': 2994.563571700699,
  'mid_upper': 3011.515992541722},
 {'title': 'CEC17 Function 27 - SPLITTING-WITH STDs',
  'mean': 2974.5718357079545,
  'lower': 2961.942903277041,
  'upper': 2987.200768138868,
  'mid_mean': 3001.755708840909,
  'mid_lower': 2989.396774314822,
  'mid_upper': 3014.114643366996},
 {'title': 'CEC17 Function 27 - SPLITTING-WITH STDs - Dynamic-split',
  'mean': 2983.779593009466,
  'lower': 2963.518626986462,
  'upper': 3004.0405590324704,
  'mid_mean': 3005.1047472304704,
  'mid_lower': 2986.743755569737,
  'mid_upper': 3

In [89]:
# create a dataframe from the result_comparision
import pandas as pd
df = pd.DataFrame(result_comparision)
df.to_csv("/Users/elotech/Desktop/DeepHiveV2/experiments/results/comparison_" + str(compare_num) +".csv")

In [90]:
df

,title,mean,lower,upper,mid_mean,mid_lower,mid_upper
0,CEC17 Function 27 - NO SPLITTING-UNFREEZE,2987.013682,2974.578222,2999.449142,3009.054367,2996.745469,3021.363266
1,CEC17 Function 27 - NO SPLITTING-DELAYED DECAY...,2973.589801,2965.058110,2982.121492,3003.039782,2994.563572,3011.515993
2,CEC17 Function 27 - SPLITTING-WITH STDs,2974.571836,2961.942903,2987.200768,3001.755709,2989.396774,3014.114643
3,CEC17 Function 27 - SPLITTING-WITH STDs - Dyna...,2983.779593,2963.518627,3004.040559,3005.104747,2986.743756,3023.465739
4,CEC17 Function 27 - SPLITTING-TWO POLICIES,2937.263102,2919.376961,2955.149244,2955.374001,2938.900271,2971.847730
5,CEC17 Function 27 - SPLITTING-TWO POLICIES - D...,2955.439319,2938.764632,2972.114007,2967.118311,2951.691155,2982.545466
6,CEC17 Function 27 - PSO,2919.646871,2902.448759,2936.844982,2935.996425,2920.486754,2951.506096


In [ ]:
from deephive.environment.optimization_functions.cec2017 import functions

In [ ]:
f1 = functions.all_functions[0]

In [ ]:
params = np.array([[-55.28, -70.43]])
f_x = f1(params)
print(f_x)

In [ ]:
ScalingHelper.scale(params, d_min=-100, d_max=100)

In [91]:
import pandas as pd

# Step 1: Define the column hierarchy
columns = [
    ('Sales', 'Q1'),
    ('Sales', 'Q2'),
    ('Inventory', 'Q1'),
    ('Inventory', 'Q2')
]

# Step 2: Create the MultiIndex
multi_index = pd.MultiIndex.from_tuples(columns, names=['Category', 'Quarter'])

# Step 3: Create the DataFrame
data = [
    [250, 265, 30, 22],  # First row of data
    [275, 260, 25, 24]   # Second row of data
]

df = pd.DataFrame(data, columns=multi_index)

print(df)


Category Sales      Inventory    
Quarter     Q1   Q2        Q1  Q2
0          250  265        30  22
1          275  260        25  24


## COMPARE EXPERIMENTS

In [92]:
import pandas as pd 

In [93]:
id = 1
result_paths = "/Users/elotech/Desktop/DeepHiveV2/experiments/results/comparison_5.csv"

In [94]:
df = pd.read_csv(result_paths)

In [96]:
df

,Unnamed: 0,title,mean,lower,upper,mid_mean,mid_lower,mid_upper
0,0,CEC17 Function 27 - NO SPLITTING-UNFREEZE,2987.013682,2974.578222,2999.449142,3009.054367,2996.745469,3021.363266
1,1,CEC17 Function 27 - NO SPLITTING-DELAYED DECAY...,2973.589801,2965.058110,2982.121492,3003.039782,2994.563572,3011.515993
2,2,CEC17 Function 27 - SPLITTING-WITH STDs,2974.571836,2961.942903,2987.200768,3001.755709,2989.396774,3014.114643
3,3,CEC17 Function 27 - SPLITTING-WITH STDs - Dyna...,2983.779593,2963.518627,3004.040559,3005.104747,2986.743756,3023.465739
4,4,CEC17 Function 27 - SPLITTING-TWO POLICIES,2937.263102,2919.376961,2955.149244,2955.374001,2938.900271,2971.847730
5,5,CEC17 Function 27 - SPLITTING-TWO POLICIES - D...,2955.439319,2938.764632,2972.114007,2967.118311,2951.691155,2982.545466
6,6,CEC17 Function 27 - PSO,2919.646871,2902.448759,2936.844982,2935.996425,2920.486754,2951.506096


In [97]:
dff = df.copy()

In [133]:
def create_benchmark_tables(dfs):
    # Step 1: Extract unique algorithm names and functions
    # Assuming algorithm names and functions can be extracted from the 'title' column
    algos = set()
    functions = set()
    for df in dfs:
        df['Function'], df['Algorithm'] = zip(*df['title'].apply(lambda x: x.split(' - ', 1)))
        algos.update(df['Algorithm'].unique())
        functions.update(df['Function'].unique())

    # Step 2: Create the MultiIndex for columns
    columns = pd.MultiIndex.from_product([sorted(algos), ['mean', 'mid_mean']], names=['Algorithm', 'Metric'])

    # Initialize an empty DataFrame to hold the aggregated data
    combined_df = pd.DataFrame(columns=columns, index=sorted(functions))

    # Step 3: Aggregate data
    for df in dfs:
        for index, row in df.iterrows():
            func = row['Function']
            algo = row['Algorithm']
            combined_df.loc[func, (algo, 'mean')] = row['mean']
            combined_df.loc[func, (algo, 'mid_mean')] = row['mid_mean']

    # Reset index name
    combined_df.index.name = 'Function'

    def highlight_max(s):
        '''
        highlight the maximum in a Series yellow.
        '''
        is_max = s == s.max()
        return ['background-color: red' if v else '' for v in is_max]

    def highlight_min(s):
        '''
        highlight the maximum in a Series yellow.
        '''
        is_min = s == s.min()
        return ['background-color: green' if v else '' for v in is_min]

    # Step 4: Display the combined DataFrame
    combined_df = combined_df.style.apply(highlight_min, axis=1)
    return combined_df


# result_comparison = create_benchmark_tables(dfs)

In [134]:
combined_df = create_benchmark_tables(dfs)

In [139]:
combined_df.to_html("/Users/elotech/Desktop/DeepHiveV2/experiments/results/comparison_5.html")

In [132]:
combined_df
# save the combined_df to a csv file
combined_df.export("/Users/elotech/Desktop/DeepHiveV2/experiments/results/comparison_5_combined.csv")

AttributeError: 'NoneType' object has no attribute 'export'

In [ ]:
combined_df.

In [95]:
# columns = ["function", "no-split", "delayed-decay", "split-stds", "split-stds-dynamic", "split-2-policies", "split-2-policies-dynamic", "vanilla-pso"]
# result_df = pd.DataFrame([], columns=columns)
# for id in range(0, 29):
#     try:
#         result_paths = f"/Users/elotech/Desktop/DeepHiveV2/experiments 2/results/result_comparison_function_{id}.csv"
#         df = pd.read_csv(result_paths)
#         function = f"f{id+1}"
#         mean_fx = df["mean"].values.tolist()
#         mean_fx = [function] + mean_fx
#         result_df.loc[len(result_df)] = mean_fx
#     except Exception as e:
#         print(e)
#         continue
